In [1]:
import sys
sys.path.append('/home/luan/futebol/Futebol')

In [2]:
from cafu.utils.queries.temp import (sem_espaco, padrao, padrao_inicio_fim, mes,
                                     jogadores_inscritos, teste_gols, gols_casa_visitante)

link do jogo: https://www.espn.com.br/futebol/partida-estatisticas?jogoId=614698

In [3]:
jogo_id='614698'

In [4]:
jogadores_casa_0, jogadores_visitante_0 = jogadores_inscritos(jogo_id)

jogadores_casa_1=[]
jogadores_visitante_1=[]
s=1
i=0
while s<=12:
    if s<12 or jogadores_casa_0[i][2]!=0:
        jogadores_casa_1.append(jogadores_casa_0[i])
    i+=1
    if jogadores_casa_0[i][2]==0:
        s+=1

s=1
i=0
while s<=12:
    if s<12 or jogadores_visitante_0[i][2]!=0:
        jogadores_visitante_1.append(jogadores_visitante_0[i])
    i+=1
    if jogadores_visitante_0[i][2]==0:
        s+=1

jogadores_casa=[]
jogadores_visitante=[]
for i in range(len(jogadores_casa_1)):
    jogadores_casa.append({'nome': jogadores_casa_1[i][0], 'camisa': jogadores_casa_1[i][1], 'inicio_jogando': jogadores_casa_1[i][2], 'final_jogando': 90})
    if i<len(jogadores_casa_1)-1 and jogadores_casa_1[i+1][2]!=0:
        jogadores_casa[i]['final_jogando']=jogadores_casa_1[i+1][2]

for i in range(len(jogadores_visitante_1)):
    jogadores_visitante.append({'nome': jogadores_visitante_1[i][0], 'camisa': jogadores_visitante_1[i][1], 'inicio_jogando': jogadores_visitante_1[i][2], 'final_jogando': 90})
    if i<len(jogadores_visitante_1)-1 and jogadores_visitante_1[i+1][2]!=0:
        jogadores_visitante[i]['final_jogando']=jogadores_visitante_1[i+1][2]

In [7]:
jogadores_casa_0

[['Boris Radunovic', 31, 0],
 ['Edoardo Goldaniga', 3, 0],
 ['Luca Ceppitelli', 23, 86],
 ['Adam Obert', 33, 0],
 ['Giorgio Altare', 15, 0],
 ['Matteo Lovato', 66, 45],
 ['Dalbert', 29, 0],
 ['Luca Gagliano', 19, 81],
 ['Razvan Marin', 8, 0],
 ['Alberto Grassi', 27, 0],
 ['Christos Kourfalidis', 39, 74],
 ['Raoul Bellanova', 12, 0],
 ['Gabriele Zappa', 25, 0],
 ['Gastón Pereiro', 20, 0],
 ['João Pedro', 10, 0],
 ['Simone Aresti', 1, 0],
 ['Eldin Lolic', 43, 0],
 ['Luigi Palomba', 41, 0],
 ['Iurie Iovu', 42, 0],
 ['Lisandru Tramoni', 45, 0],
 ['Riccardo Ladinetti', 35, 0]]

In [9]:
import requests as r
from html2json import collect
import json
from bs4 import BeautifulSoup
import datetime

In [10]:
req = r.get("https://www.espn.com.br/futebol/escalacoes?jogoId="+jogo_id)
soup=BeautifulSoup(req.content, 'html.parser')
st=soup.prettify()
abt=sem_espaco(st)

s=padrao_inicio_fim(['<span','class="name">\n','<a'],['</a>\n'],abt)

# Buscar em qual índice ocorre a mudança, dos jogadores do time da casa, para os jogadores do time visitante.
abt0=abt[s[0][0]:s[len(s)-1][1]] 
ind_change=padrao(['class="name">\n\t\t\t\n\t\t\t<span'],abt0)[0][0]+s[0][0]

player=[]
for q in range(len(s)):
    abt1=abt[s[q][0]+5:s[q][1]-1]
    abt1
    jogador = ''
    for x in abt1:
        jogador = jogador + ' ' + x

    jogador=jogador[1:-1]

    i=0
    stop=False
    while not stop:
        try:
            camisa=int(abt[s[q][0]-i][:-1])
        except:
            camisa=None
        if i==8 or camisa != None:
            stop = True
        i+=1

    abt1=abt[s[q][0]-8:s[q][0]]
    v=padrao_inicio_fim(['class="detail">\n'],['</span>\n'],abt1)
    try:
        tempo_substituicao=int(abt1[v[0][0]+1][:-2])
    except:
        try:
            tempo_substituicao=int(abt1[v[0][0]+1][:-5])
        except:
            tempo_substituicao=0

    if s[q][0] < ind_change:
        time = 'casa'
    else:
        time = 'visitante'

    player.append([jogador, camisa, tempo_substituicao, time])

jogadores_casa_0=[x[:-1] for x in player if x[-1] == 'casa']
jogadores_visitante_0=[x[:-1] for x in player if x[-1] == 'visitante']

In [12]:
from cafu.utils import WebdriverChrome

In [15]:
web = WebdriverChrome(headless=False)

In [18]:
web.web.get('https://www.espn.com.br/futebol/escalacoes?jogoId=614698')

In [21]:
links = [e.get_attribute('href') for e in web.web.find_elements_by_tag_name('a')]

In [40]:
import re
padrao = 'jogador/.+/id/[0-9]+'
a =  'https://www.espn.com.br/futebol/jogador/_/id/76017/nicolas-gonzalez'
re.findall(padrao, a)

['jogador/_/id/76017']

In [5]:
from cafu.utils import WebdriverChrome
import re

def find_id_jogadores(jogo_id):
    """
    Args:
        jogo_id: (int) completa o link https://www.espn.com.br/futebol/escalacoes?jogoId=<jogo_id>. 
    Returns:
        list: códigos dos jogadores que foram inscritos para a partida, referente ao <jogo_id>
    """
    
    web = WebdriverChrome()
    link_partida = f'https://www.espn.com.br/futebol/escalacoes?jogoId={jogo_id}'
    web.web.get(link_partida)
    
    padrao = 'jogador/.+/id/[0-9]+/.+'
    links_0 = web.web.find_elements_by_tag_name('a')
    links = [l.get_attribute('href') for l in links_0]
    
    web.web.close()
    
    ids_jogadores = []
    for l in links:
        try:
            id_ = re.findall(padrao, l)[0][13:]
            id_part1 = id_.split('/')[0]
            id_part2 = id_.split('/')[1]
            ids_jogadores.append({'part1': id_part1, 'id_part2': id_part2})
        except:
            pass
    return ids_jogadores

In [6]:
ids = find_id_jogadores(jogo_id='614698')

In [105]:
from Levenshtein import distance
from unidecode import unidecode
def identify_string_in_list(string, list_strings):
    """
    Args:
        string: (str) 
        list_strings: (list) strings 
    Returns:
        str: string na lista <list_strings> mais parecida com <string>
    """

    # lower: todas as letras minúsculas
    # unidecode: sem acento
    # replace('-',' '): substitui - por espaço
    string = unidecode(string.lower())
    list_strings = [unidecode(s.lower()).replace('-',' ') for s in list_strings] 

    index_string_in_list = 0
    min_dist = distance(string, list_strings[index_string_in_list])
    for i, s in enumerate(list_strings):
        dist = distance(string, list_strings[i])
        if dist < min_dist:
            index_string_in_list = i
            min_dist = dist
    
    #if dist > 0:
    #    logging.warning(f"WARNING : distance(string_input={string}, "
    #                    f"string_found={list_strings[index_string_in_list]}) = {dist}")
    
    return list_strings[index_string_in_list]

In [4]:
from cafu import identify_string_in_list

In [7]:
nomes_ids = [c['id_part2'] for c in ids]
nome = 'Riccardo Sottil'
#nome = 'Boris Radunovic'
identify_string_in_list(nome, nomes_ids)

'riccardo sottil'

In [ ]:
nomes_ids[]